In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random as rd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow import keras
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
import cv2
import matplotlib.pyplot as plt

**Parameters**

In [ ]:
IM_SIZE = 256

In [ ]:
import os

**Model**

In [ ]:
model = keras.models.load_model('../input/effect0-brain/Brain_flair_model_effect.h5',custom_objects={"FixedDropout": keras.layers.Dropout})

**some function**

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    #data = (data * 255).astype(np.uint8)
        
    return data
def get_prediction_per_case(patient):
    
    path = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{patient}/FLAIR/'
    list_subfolders_with_paths = [f for f in os.listdir(path)]
    
    prediction = []
    
    for images in list_subfolders_with_paths:
        
            
        img = read_xray(path+images)
                       
        if np.max(img) > 0 and np.mean(img)>= 0.015:
                
                
            img =  cv2.resize(img,(IM_SIZE,IM_SIZE))
            
            img = cv2.merge((img,img,img))
            img = tf.reshape(img, (-1, IM_SIZE, IM_SIZE, 3))
            
            pred = model.predict(img)
            
            prediction.append(pred)
    
    return np.mean(prediction,axis=0)[0][0]

In [ ]:
get_prediction_per_case('00047')

In [ ]:
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv',dtype="string")

In [ ]:
df['MGMT_value'] = df['BraTS21ID'].apply(get_prediction_per_case)

In [ ]:
df[['BraTS21ID', 'MGMT_value']].to_csv('submission.csv', index = False)

In [ ]:
df